##INSTALAMOS REPOSITORIO Y LIBRERIAS NECESARIAS

In [ ]:
from scipy.signal import resample

In [ ]:
%%capture
!pip install braindecode===0.7 ### seteamos la versión de braindecode que es con la que trabajamos en el repositorio que se carga posteriormente

In [ ]:
%%capture
from google.colab import drive ## conectamos con onedrive
drive.mount('/content/drive')
### INSTALAMOS LIBRERIAS NECESARIAS
!pip install --upgrade tensorflow
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
"""
cargamos el repositorio que es el encargado de gestionar:

-  carga de la base de datos
- preprocesamiento
- segmentación por runs
- carga del modelo de MTVAE
- entrenamiento
- gestión del metodo de validación

"""
%pip install -e git+https://github.com/UN-GCPDS/python-gcpds.MI_prediction.git#egg=MI_prediction

In [ ]:
"""
para cargar las clases si esta desde colab cambie en los archivos el nombre de la carpeta asociada al repo por
prediction es algo que no corregi por el momento, o cambie el import para el nombre de la carpeta
"""
from src.prediction.Model_Control.Global_Class.Global_Class import ModelControl
from src.prediction.Model_Control.Global_Class.Global_Class import DatasetControl

In [ ]:
### CARGAMOS LOS SUJETOS ESPECIFICOS
subjects = [1,2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25,
 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 47, 48, 50, 51, 52]

##PROBAMOS LOS SUJETOS

In [ ]:
###PROBAMOS LOS SUJETOS
acc_subj = []

#### OBTENEMOS LOS CALLBACKS NECESARIOS
callbacks_names = {'early_stopping_train1':'early_stopping','checkpoint_train1':'checkpoint',
                   'early_stopping_train2':'early_stopping','checkpoint_train2':'checkpoint',
                   'early_stopping_train3':'early_stopping','checkpoint_train3':'checkpoint',
                   'early_stopping_train4':'early_stopping','checkpoint_train4':'checkpoint'}


In [ ]:
for sbj in subjects:


    datasetControl = DatasetControl(DatasetName='GIGA') ### DEFINIMOS LA BASE DE DATOS
      ### OBTENEMOS LAS SESIONES Y RUNS DE LA BASE DE DATOS SELECCIONADA
      sessionsRuns = datasetControl.getSessionsRuns()
      X , y , sfreq = datasetControl.LoadDataset(subject = sbj,Sessions_Runs=sessionsRuns) ## SIN SEGMENTAR POR SESIONES
      print("===============================")
      print(X.shape)
      print(y.shape)
      print("===============================")
      Acc_runs = []

      # # ### GIGA TIENE 5 SESIONES USANDO EL MISMO PREPROCESO QUE MTVAE
      X_train_run_0,X_train_run_1,X_train_run_2,X_train_run_3,X_train_run_4 = X[0,0,:,:,:,:],X[0,1,:,:,:,:],X[0,2,:,:,:,:],X[0,3,:,:,:,:],X[0,4,:,:,:,:]

      y_train_run_0 =  y[0,0,:]
      y_train_run_1 =  y[0,1,:]
      y_train_run_2 =  y[0,2,:]
      y_train_run_3 =  y[0,3,:]
      y_train_run_4 =  y[0,4,:]
      #y_train_run_5 =  y[0,5,:]

      # # ### APLICAMOS RESAMPLE PARA DEFINIR LA NUEVA FRECUENCIA DE MUESTREO QUE ES LA QUE TRABAJAMOS CON EL MODELO DE EEGNET
      fs_new = 128
      X_train_run_0 = resample(X_train_run_0,int((X_train_run_0.shape[-2]/sfreq)*fs_new),axis=-2)
      X_train_run_1 = resample(X_train_run_1,int((X_train_run_1.shape[-2]/sfreq)*fs_new),axis=-2)
      X_train_run_2 = resample(X_train_run_2,int((X_train_run_2.shape[-2]/sfreq)*fs_new),axis=-2)
      X_train_run_3 = resample(X_train_run_3,int((X_train_run_3.shape[-2]/sfreq)*fs_new),axis=-2)
      X_train_run_4 = resample(X_train_run_4,int((X_train_run_4.shape[-2]/sfreq)*fs_new),axis=-2)
      #X_train_run_5 = resample(X_train_run_5,int((X_train_run_5.shape[-2]/sfreq)*fs_new),axis=-2)


      list_train = np.concatenate([X_train_run_0,X_train_run_1,X_train_run_2,X_train_run_3,X_train_run_4])

      ### definimos el número de clases
      num_classes = 2


      list_y_train = np.concatenate([y_train_run_0,y_train_run_1,y_train_run_2,y_train_run_3,y_train_run_4])


      # # ##LISTA PARA GUARDAR EL RENDIMIENTO POR RUN


      print("==============================")
      print("=========="+str(sbj)+"=============")
      print("==============================")
      # ### CARGAMOS LA BASE DE DATOS BCI2A Y EEGNET
      model_args = {'nb_classes':2,'Chans':list_train.shape[1],'Samples':list_train.shape[2],'dropoutRate':0.5}
      modelControl = ModelControl(Model = 'MTVAE_with_loss',parameters = model_args )

      # ###COMPILAMOS EL MODELO
      ### la función de costo de la KL esta definida interna en el modelo agregamos para completar el entreno el mse y categorical_crossentropy
      modelControl.compileModel(optimizer='Adam',lr=0.001,loss_list=['mse','categorical_crossentropy'],metrics = ['accuracy'])

      # ## EXPERIMENTO
      print(list_train.shape,list_y_train.shape)
      # definimos el metodo de validación
      validation = 'lawhern2018'
      # ## '/content/drive/MyDrive/EEG_DATA/EEGNET_RESULTS/EEGNET_ALL_RUNS_BI_GIGA/'+
      call_args = [
          {'monitor':'val_out_activation_accuracy','patience':100,'min_delta':0.001,'mode':'max','verbose':1,'restore_best_weights':False},
          {'filepath':'/content/drive/MyDrive/EEG_DATA/MTVAE_GIGA/'+'checkpoint1_'+str(sbj),'save_format':'tf','monitor':'val_out_activation_accuracy','verbose':1,'save_best_only':True,'save_weights_only':True},
          {'monitor':'val_out_activation_accuracy','patience':100,'min_delta':0.001,'mode':'max','verbose':1,'restore_best_weights':False},
          {'filepath':'/content/drive/MyDrive/EEG_DATA//MTVAE_GIGA/'+'checkpoint2_'+str(sbj),'save_format':'tf','monitor':'val_out_activation_accuracy','verbose':1,'save_best_only':True,'save_weights_only':True},
          {'monitor':'val_out_activation_accuracy','patience':100,'min_delta':0.001,'mode':'max','verbose':1,'restore_best_weights':False},
          {'filepath':'/content/drive/MyDrive/EEG_DATA//MTVAE_GIGA/'+'checkpoint3_'+str(sbj),'save_format':'tf','monitor':'val_out_activation_accuracy','verbose':1,'save_best_only':True,'save_weights_only':True},
          {'monitor':'val_out_activation_accuracy','patience':100,'min_delta':0.001,'mode':'max','verbose':1,'restore_best_weights':False},
          {'filepath':'/content/drive/MyDrive/EEG_DATA//MTVAE_GIGA/'+'checkpoint4_'+str(sbj),'save_format':'tf','monitor':'val_out_activation_accuracy','verbose':1,'save_best_only':True,'save_weights_only':True},
      ]

      History,acc_model  = modelControl.train_model(X_train=list_train,Y_train=[list_train,list_y_train],x_val=list_train,y_val=[list_train,list_y_train],callbacks_names = callbacks_names,call_args =call_args,validation_mode = validation, batch_size =16,epochs = 500,verbose =1,autoencoder=True)

      print("ACCURACY PER SUBJECT: ",acc_model)
      acc_subj.append(acc_model)



In [ ]:
### RESULTADOS OBTENIDOS
acc_subj = [89.0,
 66.5,
 98.0,
 90.0,
 99.0,
 84.0,
 72.0,
 79.5,
 77.5,
 90.5,
 73.5,
 75.5,
 96.5,
 98.0,
 83.5,
 76.5,
 78.5,
 78.0,
 80.5,
 75.0,
 63.5,
 82.0,
 95.5,
 78.0,
 87.0,
 96.5,
 78.5,
 90.0,
 81.0,
 78.0,
 79.0,
 79.0,
 70.5,
 85.5,
 78.0,
 87.0,
 66.5,
 80.0,
 77.5,
 96.0,
 79.5,
 94.0,
 95.0,
 75.0,
 83.0,
 96.0,
 100.0,
 76.5,
 78.5]

In [ ]:
len(acc_subj)

In [ ]:
len(subjects)

##MIRAMOS POR RUNS PARA CADA SUJETO

In [ ]:
Acc__subs_runs = []
# subjects=[43]

In [ ]:
for sbj in [51,52]:

    datasetControl = DatasetControl(DatasetName='GIGA') ### DEFINIMOS LA BASE DE DATOS
    ### OBTENEMOS LAS SESIONES Y RUNS DE LA BASE DE DATOS SELECCIONADA
    sessionsRuns = datasetControl.getSessionsRuns()
    X , y , sfreq = datasetControl.LoadDataset(subject = sbj,Sessions_Runs=sessionsRuns) ## SIN SEGMENTAR POR SESIONES

    Acc_runs = []
    # ### BCI2A TIENE 2 SESIONES USANDO EL MISMO PREPROCESO QUE MTVAE
    X_train_run_0,X_train_run_1,X_train_run_2,X_train_run_3,X_train_run_4 = X[0,0,:,:,:,:],X[0,1,:,:,:,:],X[0,2,:,:,:,:],X[0,3,:,:,:,:],X[0,4,:,:,:,:]

    y_train_run_0 =  y[0,0,:]
    y_train_run_1 =  y[0,1,:]
    y_train_run_2 =  y[0,2,:]
    y_train_run_3 =  y[0,3,:]
    y_train_run_4 =  y[0,4,:]
    #y_train_run_5 =  y[0,5,:]


    # ### APLICAMOS RESAMPLE
    fs_new = 128
    X_train_run_0 = resample(X_train_run_0,int((X_train_run_0.shape[-2]/sfreq)*fs_new),axis=-2)
    X_train_run_1 = resample(X_train_run_1,int((X_train_run_1.shape[-2]/sfreq)*fs_new),axis=-2)
    X_train_run_2 = resample(X_train_run_2,int((X_train_run_2.shape[-2]/sfreq)*fs_new),axis=-2)
    X_train_run_3 = resample(X_train_run_3,int((X_train_run_3.shape[-2]/sfreq)*fs_new),axis=-2)
    X_train_run_4 = resample(X_train_run_4,int((X_train_run_4.shape[-2]/sfreq)*fs_new),axis=-2)
    #X_train_run_5 = resample(X_train_run_5,int((X_train_run_5.shape[-2]/sfreq)*fs_new),axis=-2)


    list_train = [X_train_run_0,X_train_run_1,X_train_run_2,X_train_run_3,X_train_run_4]

    num_classes = 2


    list_y_train = [y_train_run_0,y_train_run_1,y_train_run_2,y_train_run_3,y_train_run_4]


    # ##LISTA PARA GUARDAR EL RENDIMIENTO POR RUN

    for i in range(0,len(list_train)):

        print("==============================")
        print("=========="+str(sbj)+'RUN'+str(i+1)+"=============")
        print("==============================")
        ### CARGAMOS LA BASE DE DATOS BCI2A Y EEGNET
        model_args = {'nb_classes':2,'Chans':list_train[i].shape[1],'Samples':list_train[i].shape[2],'dropoutRate':0.5}
        modelControl = ModelControl(Model = 'MTVAE_with_loss',parameters = model_args )

        # ###COMPILAMOS EL MODELO
        modelControl.compileModel(optimizer='Adam',lr=0.001,loss_list=['mse','categorical_crossentropy'],metrics = ['accuracy'])

        ## EXPERIMENTO

        validation = 'lawhern2018'
        ## '/content/drive/MyDrive/EEG_DATA/EEGNET_RESULTS/EEGNET_ALL_RUNS_BI_GIGA/'+
        call_args = [
         {'monitor':'val_out_activation_accuracy','patience':100,'min_delta':0.001,'mode':'max','verbose':1,'restore_best_weights':False},
         {'filepath':'/content/drive/MyDrive/EEG_DATA/MTVAE_GIGA/sbj_'+str(sbj)+'_Run_'+str(i+1)+'_'+'checkpoint1','save_format':'tf','monitor':'val_out_activation_accuracy','verbose':1,'save_best_only':True,'save_weights_only':True},
         {'monitor':'val_out_activation_accuracy','patience':100,'min_delta':0.001,'mode':'max','verbose':1,'restore_best_weights':False},
         {'filepath':'/content/drive/MyDrive/EEG_DATA/MTVAE_GIGA/sbj_'+str(sbj)+'_Run_'+str(i+1)+'_'+'checkpoint2','save_format':'tf','monitor':'val_out_activation_accuracy','verbose':1,'save_best_only':True,'save_weights_only':True},
         {'monitor':'val_out_activation_accuracy','patience':100,'min_delta':0.001,'mode':'max','verbose':1,'restore_best_weights':False},
         {'filepath':'/content/drive/MyDrive/EEG_DATA/MTVAE_GIGA/sbj_'+str(sbj)+'_Run_'+str(i+1)+'_'+'checkpoint3','save_format':'tf','monitor':'val_out_activation_accuracy','verbose':1,'save_best_only':True,'save_weights_only':True},
         {'monitor':'val_out_activation_accuracy','patience':100,'min_delta':0.001,'mode':'max','verbose':1,'restore_best_weights':False},
         {'filepath':'/content/drive/MyDrive/EEG_DATA/MTVAE_GIGA/sbj_'+str(sbj)+'_Run_'+str(i+1)+'_'+'checkpoint4','save_format':'tf','monitor':'val_out_activation_accuracy','verbose':1,'save_best_only':True,'save_weights_only':True},
        ]

        History,acc_model  = modelControl.train_model(X_train=list_train[i],Y_train=[list_train[i],list_y_train[i]],x_val=list_train[i],y_val=[list_train[i],list_y_train[i]],callbacks_names = callbacks_names,call_args =call_args,validation_mode = validation, batch_size =16,epochs = 500,verbose =1,autoencoder=True)

        Acc_runs.append(acc_model)
    print("////////////////////////////")
    print("////////////////////////////")
    print("SUBJECT RUNS: ",Acc_runs)
    print("////////////////////////////")
    print("////////////////////////////")
    Acc__subs_runs.append(Acc_runs)

In [ ]:
Acc__subs_runs